In [1]:
import sys
from pathlib import Path
import shutil
import json
import csv

from shapely.geometry import Point
import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

TMP = DIR/'tmp'
IN_DIR = TMP/'inputs'
OUT_DIR = TMP/'outputs'

%load_ext autoreload
%autoreload 2



In [2]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# collection = {
#     'type': 'FeatureCollection', 
#     'features': list(map(wt.build_feature, tids))
#     }
# json.dumps(collection)

In [3]:
# # Download SRTM data from NASA

# path = Path('/home/araichev/srtmnz/srtm1')
# url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL3.003/2000.02.11/"
# #url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/"
# tids = wt.SRTM_NZ_TILE_IDS
# for i, tid in enumerate(sorted(tids)):
#     filename = tid + '.SRTMGL3.hgt.zip'
#     href = url + filename
#     print(i, href)

In [4]:
# # Download elevation test data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_tile_names(lon_lats)
# print('Tiles to download =', tiles)
# wt.download_srtm(tiles, TEST_DATA_DIR/'srtm3', api_key=wt.get_secret('GITLAB_API_KEY'))

In [5]:
# Start fresh
wt.rm_paths(TMP)


In [6]:
high_definition = True

path = DIR/'tests'/'data'/'transmitters.csv'
transmitters = wt.read_transmitters(path)
print('num transmitters =', len(transmitters))
for t in transmitters:
    print(t)

num transmitters = 20
{'vertical_beamwidth': '', 'power_eirp': 4.0, 'site_name': '4', 'antenna_height': 10.0, 'name': 'GrayWireless_4', 'horizontal_beamwidth': '', 'longitude': 174.336547, 'polarization': 0.0, 'downtilt': '', 'frequency': 5725.0, 'latitude': -35.712247, 'network_name': 'Gray Wireless', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '5', 'antenna_height': 20.0, 'name': 'MyNetwork_5', 'horizontal_beamwidth': '90', 'longitude': 174.281534, 'polarization': 0.0, 'downtilt': '1', 'frequency': 5725.0, 'latitude': -35.658841, 'network_name': 'My Network', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '6', 'antenna_height': 20.0, 'name': 'MyNetwork_6', 'horizontal_beamwidth': '90', 'longitude': 173.156248, 'polarization': 0.0, 'downtilt': '1', 'frequency': 5725.0, 'latitude': -34.819217, 'network_name': 'My Network', 'bearing': '0'}
{'vertical_beamwidth': '30', 'power_eirp': 4.0, 'site_name': '7', 'antenna_height': 20

In [ ]:
# Create SPLAT transmitter files

in_path = DIR/'tests'/'data'/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
if out_path.exists():
    shutil.rmtree(str(out_path))
    
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

CanUCMe_21.az       MyNetwork_11.az   MyNetwork_9.az   Skynet_14.az
CanUCMe_21.el       MyNetwork_11.el   MyNetwork_9.el   Skynet_14.el
CanUCMe_21.lrp      MyNetwork_11.lrp  MyNetwork_9.lrp  Skynet_14.lrp
CanUCMe_21.qth      MyNetwork_11.qth  MyNetwork_9.qth  Skynet_14.qth
CanUCMe_22.az       MyNetwork_5.az    RuralCom_18.az   Skynet_15.az
CanUCMe_22.el       MyNetwork_5.el    RuralCom_18.el   Skynet_15.el
CanUCMe_22.lrp      MyNetwork_5.lrp   RuralCom_18.lrp  Skynet_15.lrp
CanUCMe_22.qth      MyNetwork_5.qth   RuralCom_18.qth  Skynet_15.qth
CanUCMe_23.az       MyNetwork_6.az    RuralCom_19.az   Skynet_16.az
CanUCMe_23.el       MyNetwork_6.el    RuralCom_19.el   Skynet_16.el
CanUCMe_23.lrp      MyNetwork_6.lrp   RuralCom_19.lrp  Skynet_16.lrp
CanUCMe_23.qth      MyNetwork_6.qth   RuralCom_19.qth  Skynet_16.qth
GrayWireless_4.az   MyNetwork_7.az    Skynet_12.az     Skynet_17.az
GrayWireless_4.el   MyNetwork_7.el    Skynet_12.el     Skynet_17.el
GrayWireless_4.lrp  MyNetwork_7.lrp   Skyn

In [ ]:
# Download topography data

points = [Point(p) for p in wt.get_lonlats(transmitters)]
tids = wt.compute_tile_cover(list(points))
print('Tiles to download =', tids)

if high_definition:
    path = IN_DIR/'srtm1'
else:
    path = IN_DIR/'srtm3'
api_key = wt.get_secret('GITLAB_API_KEY')

wt.download_srtm(tids, path, high_definition=high_definition, api_key=api_key)

% ls {path}

Tiles to download = ['S35E173', 'S36E173', 'S36E174', 'S37E174', 'S37E175', 'S38E174', 'S38E175', 'S38E176', 'S40E176', 'S41E175', 'S42E175', 'S46E168', 'S47E167']


In [ ]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'srtm1'
else:
    in_path = IN_DIR/'srtm3'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

In [ ]:
# Create coverage reports

in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitters=transmitters, 
  high_definition=high_definition)

% ls {out_path}

In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path)

% ls {path}